<a href="https://colab.research.google.com/github/TonyWang126/NYU-Econometrics-2021/blob/main/Note_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Briefings**

1. Two packages:


*   https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html
*   https://bashtage.github.io/linearmodels/doc/iv/examples/advanced-examples.html

2. Four variables:

Dependent Variables: Y
Endogenous Variable: X
Control variables: W
instruments: Z

3. Two regressions


*   First: X~Z
*   Second: Y~(estimated_X+W)





background readings: https://cryptobriefing.com/earning-20-apy-with-yield-app/

# Data Preparations

### Date Sources 
https://docs.defipulse.com/overview-of-api-endpoints

https://data-beta.defipulse.com/dashboard/egs

https://docs.defipulse.com/api-docs-by-provider/defi-pulse-data/total-value-locked/total-value-locked

### Import the Historical Data of Total Value Locked

In [43]:
import pandas as pd
import requests
import json 
from datetime import datetime
TVL= requests.get('https://data-api.defipulse.com/api/v1/defipulse/api/GetHistory?resolution=history&project=all&api-key=05f8e579a2bec58c6fb86b5ad1d84b76b0267a190307cdf6f194960c4121')
TVL= TVL.json()
TVL=pd.DataFrame(TVL)
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI
0,1624179600,55152945101,2.492822e+07,171912.219296,8.635372e+06,2.833782e+09
1,1624147200,54627070780,2.506312e+07,172820.592158,8.654305e+06,2.835322e+09
2,1624060800,55438570398,2.478987e+07,172546.542025,8.676121e+06,2.921278e+09
3,1623974400,58737024010,2.473284e+07,169897.285278,8.633628e+06,2.909253e+09
4,1623888000,59366500781,2.506841e+07,174916.777142,8.701187e+06,3.120715e+09


### Epoch & Unix Timestamp Conversion Tools: 

https://www.epochconverter.com/

In [44]:
import datetime
TVL['timestamp']=TVL['timestamp'].astype(int)
TVL.dtypes

timestamp      int64
tvlUSD         int64
tvlETH       float64
BTC          float64
ETH          float64
DAI          float64
dtype: object

In [45]:
TVL['time']=TVL['timestamp'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).replace(tzinfo=datetime.timezone.utc))
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time
0,1624179600,55152945101,2.492822e+07,171912.219296,8.635372e+06,2.833782e+09,2021-06-20 09:00:00+00:00
1,1624147200,54627070780,2.506312e+07,172820.592158,8.654305e+06,2.835322e+09,2021-06-20 00:00:00+00:00
2,1624060800,55438570398,2.478987e+07,172546.542025,8.676121e+06,2.921278e+09,2021-06-19 00:00:00+00:00
3,1623974400,58737024010,2.473284e+07,169897.285278,8.633628e+06,2.909253e+09,2021-06-18 00:00:00+00:00
4,1623888000,59366500781,2.506841e+07,174916.777142,8.701187e+06,3.120715e+09,2021-06-17 00:00:00+00:00


In [46]:
TVL['date'] = pd.to_datetime(TVL['time']).dt.date
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time,date
0,1624179600,55152945101,2.492822e+07,171912.219296,8.635372e+06,2.833782e+09,2021-06-20 09:00:00+00:00,2021-06-20
1,1624147200,54627070780,2.506312e+07,172820.592158,8.654305e+06,2.835322e+09,2021-06-20 00:00:00+00:00,2021-06-20
2,1624060800,55438570398,2.478987e+07,172546.542025,8.676121e+06,2.921278e+09,2021-06-19 00:00:00+00:00,2021-06-19
3,1623974400,58737024010,2.473284e+07,169897.285278,8.633628e+06,2.909253e+09,2021-06-18 00:00:00+00:00,2021-06-18
4,1623888000,59366500781,2.506841e+07,174916.777142,8.701187e+06,3.120715e+09,2021-06-17 00:00:00+00:00,2021-06-17


In [47]:
TVL=TVL.drop(columns='time')
TVL['date']=pd.to_datetime(TVL['date'])
TVL.dtypes

timestamp             int64
tvlUSD                int64
tvlETH              float64
BTC                 float64
ETH                 float64
DAI                 float64
date         datetime64[ns]
dtype: object

### Import the data for Ethereum Prices

[Data Metrics](https://github.com/coinmetrics-io/data/blob/master/csv/metrics.csv)

In [48]:
df_eth=pd.read_csv("https://raw.githubusercontent.com/coinmetrics-io/data/master/csv/eth.csv",usecols=(["time","PriceUSD","CapMrktCurUSD","TxTfrValAdjUSD","AdrBalCnt","NVTAdj90","SplyCur","FlowOutExUSD"]))
df_eth['time']=pd.to_datetime(df_eth['time'])
df_eth.dtypes

time              datetime64[ns]
AdrBalCnt                float64
CapMrktCurUSD            float64
FlowOutExUSD             float64
NVTAdj90                 float64
PriceUSD                 float64
SplyCur                  float64
TxTfrValAdjUSD           float64
dtype: object

### Merge the two Data Frame

In [49]:
df =df_eth.merge(TVL, left_on='time', right_on='date')
df.head()

,time,AdrBalCnt,CapMrktCurUSD,FlowOutExUSD,NVTAdj90,PriceUSD,SplyCur,TxTfrValAdjUSD,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,date
0,2017-08-02,2563959.0,2.039684e+10,7.162628e+07,20.362229,217.584664,9.374206e+07,6.780871e+08,1501632000,0,0.000000,0.0,0.0,0.0,2017-08-02
1,2017-08-03,2583277.0,2.100563e+10,9.191231e+07,20.979627,224.028898,9.376303e+07,6.062566e+08,1501718400,4,0.018339,0.0,0.0,0.0,2017-08-03
2,2017-08-04,2605667.0,2.077052e+10,7.497462e+07,20.714810,221.471094,9.378433e+07,5.206240e+08,1501804800,4,0.017826,0.0,0.0,0.0,2017-08-04
3,2017-08-05,2624386.0,2.335192e+10,1.051840e+08,23.205816,248.939666,9.380552e+07,6.665590e+08,1501891200,5,0.022665,0.0,0.0,0.0,2017-08-05
4,2017-08-06,2640032.0,2.496085e+10,1.046373e+08,24.667311,266.031146,9.382679e+07,9.229966e+08,1501977600,5,0.019756,0.0,0.0,0.0,2017-08-06


In [50]:
df=df[["date","tvlUSD","PriceUSD","SplyCur","AdrBalCnt","FlowOutExUSD"]]

# Test Instrument Validities

Y: "tvlUSD"

X: "FlowOutExUSD"

Z: "SplyCur"

W: "AdrBalCnt"

In [51]:
df[["FlowOutExUSD","SplyCur"]].corr()

,FlowOutExUSD,SplyCur
FlowOutExUSD,1.000000,0.312248
SplyCur,0.312248,1.000000


# Using the Linear Model Package

https://bashtage.github.io/linearmodels/doc/iv/index.html

In [42]:
pip install linearmodels

In [52]:
from linearmodels import IV2SLS

### run an OLS

In [53]:
df['const'] = 1

In [54]:
ivolsmod = IV2SLS(df.tvlUSD, df[["const","FlowOutExUSD","SplyCur","AdrBalCnt"]], None, None)
res_ols = ivolsmod.fit()
print(res_ols)

                            OLS Estimation Summary                            
Dep. Variable:                 tvlUSD   R-squared:                      0.8326
Estimator:                        OLS   Adj. R-squared:                 0.8322
No. Observations:                1417   F-statistic:                    1815.6
Date:                Sun, Jun 20 2021   P-value (F-stat)                0.0000
Time:                        09:49:42   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
const          3.79e+11  2.965e+10     12.781     0.0000   3.209e+11   4.371e+11
FlowOutExUSD     19.529     1.6356     11.94

### run an IV2SLS

In [55]:
ivmod = IV2SLS(df.tvlUSD, df[["const","AdrBalCnt"]], df["FlowOutExUSD"], df["SplyCur"])
res_2sls = ivmod.fit()
print(res_2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 tvlUSD   R-squared:                      0.6971
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6967
No. Observations:                1417   F-statistic:                    835.24
Date:                Sun, Jun 20 2021   P-value (F-stat)                0.0000
Time:                        09:49:42   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                               
              Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------
const        -1.201e+10  5.804e+08    -20.687     0.0000  -1.314e+10  -1.087e+10
AdrBalCnt        306.11     16.772     18.25

### comparison of the results

In [56]:
from collections import OrderedDict
from linearmodels.iv.results import compare

In [57]:
res = OrderedDict()
res['OLS'] = res_ols
res['2SLS'] = res_2sls
print(compare(res))

                 Model Comparison                
                               OLS           2SLS
-------------------------------------------------
Dep. Variable               tvlUSD         tvlUSD
Estimator                      OLS        IV-2SLS
No. Observations              1417           1417
Cov. Est.                   robust         robust
R-squared                   0.8326         0.6971
Adj. R-squared              0.8322         0.6967
F-statistic                 1815.6         835.24
P-value (F-stat)            0.0000         0.0000
==================     ===========   ============
const                     3.79e+11     -1.201e+10
                          (12.781)      (-20.687)
FlowOutExUSD                19.529         41.777
                          (11.940)       (24.217)
SplyCur                    -4144.4               
                         (-13.135)               
AdrBalCnt                   2129.5         306.11
                          (15.107)       (18.251)


### Testing endogeneity

The Durbin test is a classic of endogeneity which compares OLS estimates with 2SLS and exploits the fact that OLS estimates will be relatively efficient. Durbin’s test is not robust to heteroskedasticity.

In [58]:
res_2sls.durbin()

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 313.6836
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7ff88881a7d0

Wooldridge’s regression-based test of exogeneity is robust to heteroskedasticity since it inherits the covariance estimator from the model. Here there is little difference.

In [59]:
res_2sls.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 172.5277
P-value: 0.0000
Distributed: chi2(1)
WaldTestStatistic, id: 0x7ff88891a190

### print first stage regression

In [60]:
print(res_2sls.first_stage)

     First Stage Estimation Results    
                           FlowOutExUSD
---------------------------------------
R-squared                        0.4602
Partial R-squared                0.3515
Shea's R-squared                 0.3515
Partial F-statistic              398.75
P-value (Partial F-stat)         0.0000
Partial F-stat Distn            chi2(1)
==========================  ===========
const                         1.757e+10
                               (20.168)
AdrBalCnt                        81.956
                               (19.500)
SplyCur                         -186.29
                              (-19.969)
---------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


# Using the Statsmodel Package

Not recommended

https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html